In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelDuplicatingPixelUnshuffleUpSampleLayer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        factor: int,
    ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.factor = factor
        assert out_channels * factor**2 % in_channels == 0
        self.repeats = out_channels * factor**2 // in_channels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.repeat_interleave(self.repeats, dim=1)
        x = F.pixel_shuffle(x, self.factor)
        return x

class PixelUnshuffleChannelAveragingDownSampleLayer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        factor: int,
    ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.factor = factor
        assert in_channels * factor**2 % out_channels == 0
        self.group_size = in_channels * factor**2 // out_channels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.pixel_unshuffle(x, self.factor)
        B, C, H, W = x.shape
        x = x.view(B, self.out_channels, self.group_size, H, W)
        x = x.mean(dim=2)
        return x
    
# Input parameters
in_channels = 32
out_channels = 64
factor = 2

# Create the layer
downsample_layer = PixelUnshuffleChannelAveragingDownSampleLayer(in_channels, out_channels, factor)
upsample_layer = ChannelDuplicatingPixelUnshuffleUpSampleLayer(out_channels,in_channels,factor)

# Generate a random tensor with shape (B, in_channels, H, W)
B, H, W = 1, 32, 64
x = torch.randn(B, in_channels, H, W)

# Forward pass through the layer
output = downsample_layer(x)
up = upsample_layer(output)

# Print the output shape and content
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f'up shape:{up.shape}')


In [ ]:
import torch

# 创建一个需要梯度的张量
src_tensor = torch.randn(3, 3, requires_grad=True)
print("Source tensor requires_grad:", src_tensor.requires_grad)  # True

# 创建一个不需要梯度的张量
target_tensor = torch.randn(3, 3, requires_grad=False)
print("Target tensor requires_grad before:", target_tensor.requires_grad)  # False

# 将src_tensor赋值给target_tensor
target_tensor = src_tensor  # 使用 clone 方法复制数据
print("Target tensor requires_grad after:", target_tensor.requires_grad)  # True


In [ ]:
import numpy as np

# 创建一个 32x64 的随机图像
image = np.random.rand(32, 64)

# 计算图像的均值
mean_value = np.mean(image)

# 将图像分为 2x2 块
blocks = image.reshape(16, 2, 32, 2).swapaxes(1, 2)

# 总块数
total_blocks = 16 * 32

# 计算需要替换的块数
num_blocks_to_replace = int(total_blocks * 0.3)

# 随机选择块
indices_to_replace = np.random.choice(total_blocks, num_blocks_to_replace, replace=False)

# 将选中的块的所有值设置为均值
for idx in indices_to_replace:
    row = idx // 32
    col = idx % 32
    blocks[row, col, :, :] = mean_value

# 重新组合图像
masked_image = blocks.swapaxes(1, 2).reshape(32, 64)

# 显示结果（使用 matplotlib 展示图像）
import matplotlib.pyplot as plt

plt.imshow(masked_image, cmap='gray')
plt.title("Masked Image")
plt.colorbar()
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt

def mask_replace_blocks(tensor):
    # Calculate the mean of each channel across the entire batch
    channel_means = tensor.mean(dim=(0, 1, -2, -1), keepdim=True)

    # Get basic dimensions of the tensor
    b, t, c, h, w = tensor.shape

    # Generate a random mask for 2x2 blocks
    block_mask = torch.rand(b, t, c, h//2, w//2) < 0.3  # 30% probability

    # Expand the block mask to cover the entire dimensions of the blocks
    mask = block_mask.repeat(1, 1, 1, 2, 2)

    # Broadcast the channel means across the tensor dimensions
    channel_means_expanded = channel_means.repeat(b, t, 1, h, w)

    # Apply the mask and replace the selected blocks
    tensor[mask] = channel_means_expanded[mask]

    return tensor, mask

# Generate a sample tensor with dimensions (2, 3, 4, 32, 64)
input_tensor = torch.rand(2, 3, 4, 32, 64)

# Apply the mask replace function
masked_tensor, mask = mask_replace_blocks(input_tensor.clone())

# Plotting
fig, axs = plt.subplots(2 * 3 * 4, 3, figsize=(15, 60))

for i in range(2):  # Batch size
    for j in range(3):  # Time steps
        for k in range(4):  # Channels
            idx = i * 3 * 4 + j * 4 + k
            # Original image
            axs[idx, 0].imshow(input_tensor[i, j, k].numpy(), cmap='gray')
            axs[idx, 0].axis('off')
            axs[idx, 0].set_title(f'Original - Batch {i}, Time {j}, Channel {k}')

            # Masked image
            axs[idx, 1].imshow(masked_tensor[i, j, k].numpy(), cmap='gray')
            axs[idx, 1].axis('off')
            axs[idx, 1].set_title(f'Masked - Batch {i}, Time {j}, Channel {k}')

            # Masked image
            axs[idx, 2].imshow(mask[i, j, k].numpy(), cmap='gray')
            axs[idx, 2].axis('off')
            axs[idx, 2].set_title(f'mask - Batch {i}, Time {j}, Channel {k}')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# 定义输入和输出索引范围
idx_in = np.array([-3, -2, -1, 0])
idx_out = np.array([1, 2, 3, 4])

# 计算不重叠的步长
sequence_length = len(idx_in) + len(idx_out)  # 总序列长度为8

# 从索引11开始，以步长为 sequence_length 生成 valid_idx
valid_idx = np.arange(3, 21 - 4, 1)
valid_idx